In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install tabulate

In [0]:
!pip install gensim

In [0]:
import nltk
nltk.download('wordnet')
nltk.download("stopwords")

In [0]:
import sys


%matplotlib inline

# basic python libraries

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from collections import Counter, defaultdict

from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedShuffleSplit

from sklearn.pipeline import Pipeline

from tabulate import tabulate

import struct

# NLP /related /libraries

from gensim.models.word2vec import Word2Vec
from nltk.stem import PorterStemmer
import regex as re


from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import SnowballStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


lmt = WordNetLemmatizer()
stemmer = SnowballStemmer("english")
ps = PorterStemmer()


stop_words = stopwords.words('english')


train_data = pd.read_csv("gdrive/My Drive/Colab Notebooks/train.csv")

test_data = pd.read_csv("gdrive/My Drive/test.csv")

y = pd.read_csv("gdrive/My Drive/Colab Notebooks/train.csv").Label



train_tweets = train_data.TweetText[:]
test_tweets = test_data.TweetText[:]

In [59]:
FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text) :
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = " {} ".format(hashtag_body.lower())
    else:
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

  
def allcaps(text) :
    text = text.group()
    return text.lower() + " <allcaps>"
  
def apostr(s) :
  if s[0]=="\'" :
    return s[1:]
  elif s[len(s)-1]=="\'" :
    return s[:len(s)-1]
  else :
    return s


def tokenize(text):
    # Different regex parts for smiley faces
    
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    
    #text = re_sub(r"@\w+", " ")  # removing this ones gave a good accuracy boost
    
    
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    
    
    
    text = re_sub(r"/"," / ")
    
    text = re_sub(r"<3","love")
    
    text = re_sub(r"\\"," ")
    
    # numbers / hashtags / elong / allcaps
    
    
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")
    text = re_sub(r"([A-Z]){2,}", allcaps)
    
    # what I aded
    
    text = re_sub(r"[\,\:\!\@\#\$\%\^\&\*\(\)\_\+\|\~\=\-\{\}\[\]\?\/\.\"\;`]", " ") # punc
    
    
    #text = re_sub(r"[^\w\s\<\>\']", " ")

    text = re_sub(r"[\<]", " <")
    text = re_sub(r"[\>]", "> ")
    
    # dealing with \'
    
    text = re_sub(r"\'ll", " will ") # treating  i'll and we'll ..
    text = re_sub(r"\'s", " ")
    text = re_sub(r"\'m", " am")
    text = re_sub(r"\'d", " would")
    text = re_sub(r"\'ve", " have")
    text = re_sub(r"\'re", " are")
    text = re_sub(r"can't", " can not")
    text = re_sub(r"\'t", " not")
    text = re_sub(r"fc "," football club")
    
    
    return text.lower()
  
  
  
  
def tokenize_(liste_tweets) :
  
  tokens = [re.findall(r"\S+",tokenize(i)) for i in liste_tweets]
  
  
  # here should happen the mapping :
  
  
  # lematizing / steming / stop words
  
  for i in range(len(tokens)) :
      #tokens[i] = [ lmt.lemmatize(j) for j in tokens[i] ] # lemmatizing    
      #tokens[i] = [ ps.stem(j) for j in tokens[i] ] # stemming    
      tokens[i] = [ j for j in tokens[i] if j not in stop_words ] # deleting stop words should make a list words specified to twitter 
      
      
  
  
  # dealing with \'
  
  for i in range(len(tokens)) :
    for j in range(len(tokens[i])) :
      if "\'" in tokens[i][j] :
        tokens[i][j] = apostr(tokens[i][j])
  
  # cleaning some noise :
  
  for i in range(len(tokens))  :
      tokens[i] = [k for k in tokens[i] if len(k)>0 ]
      
  
  
  return tokens


  

X = tokenize_(train_tweets) 


  

test_tokens = tokenize_(test_tweets) 


map_words(X)


  

map_words(test_tokens)


X_ = X + test_tokens


mult_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
#bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x,ngram_range=(1,1))), ("bernoulli nb", BernoulliNB())])
mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
#bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x,ngram_range=(1,1))), ("bernoulli nb", BernoulliNB())])
#svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x,ngram_range=(1,1))), ("linear svc", SVC(C=10,gamma=1))])
#svc_ = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x,ngram_range=(1, 3))), ("linear svc", SVC(C=10,gamma=1))])


all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
]
"""("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    ("svc_tfidf", svc_tfidf)"""

unsorted_scores = [(name, cross_val_score(model, X, y, cv=10).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])


print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model            score
-------------  -------
mult_nb         0.9629
mult_nb_tfidf   0.9603


In [0]:
test_tokens

In [0]:
def submit_(string_,clf) :
  test_id = test_data.TweetId
  clf.fit(X,y)
  prediction = clf.predict(test_tokens)
  df = pd.DataFrame(data={"TweetId":test_id,"Label":prediction})
  df.to_csv(string_,index=False)

In [0]:
submit_("sub_with_mapping_test_tokens2.csv",mult_nb)

In [0]:
def find_s(X_,str_,int_) :
  for i in range(len(X_)) :
    for j in X_[i] :
      if str_ in j :
        if int_ == 1 :
          if j == str_ :
            if i<len(X) :
               print(j," >>>>> ",y[i])
            else :
              print(j," >>>>> ")          
        else : 
          if i<len(X) :
            print(j," >>>>> ",y[i])
          else :
            print(j," >>>>> ")

In [0]:
find_s(X_,"mcfc",0)

In [0]:
def predict_(X,y) :
  return cross_val_score(mult_nb,X,y,cv=5).mean()

In [0]:
xtr,xte,ytr,yte = train_test_split(X,y,test_size=0.2)


dictio = {}

yy = list(yte)

predict_pro = [0 for i in xte]

for i in range(len(xte)) :
  dictio[i]=0
  
err_gen = []


def predict_2(clf) :
  err = 0
  clf.fit(xtr,ytr)
  
  pred = clf.predict(xte)
  pred2 = clf.predict_proba(xte)
  
  # proba = clf.predict_proba(xte)
  for i in range(len(xte)) :             
    if yy[i] != pred[i] :
      dictio[i]+=1
  return pred2

models = [
     mult_nb,
     mult_nb_tfidf
]


for i in models :
  pred2 = predict_2(i)
  
  predict_pro = [ pred2[i] + predict_pro[i] for i in range(len(predict_pro)) ]
t1 = []

for i in dictio.keys() :
  if dictio[i]==2 :
    t1 += [i]
    print(i , yy[i] , xte[i])

In [55]:
find_s(X_,"mcfc",0)

mcfci  >>>>> 


In [0]:
dic_map = {
           "sports":["sports"],
           "football":["sports","football","basketball"],
           "ausg":["ausgp","sports","gp","australia"],
           "tennis":["tennis","sports"],
           "olympics":["sports","olympics"],
           "espn":["espn","sports"],
           "cricket":["cricket","sports"],
           "fans" :["fans","sport"],
            
          "club":["club","sports"],
    
            "mcfc":["mcfc",'manchester', 'city', 'football', 'club'],
           "cfc":["cfc","chelsea","football" , "club","sports"],
           "lfc":["lfc","liverpool","football" , "club","sports"],
         
           "motogp":["gp","sports","motogp"],
    
            "parli" :["parliament","politics"],
           "obama":["politics","obama","president","gov"],
           "clinton":["politics","obama","politics","clinton"],
           "gov" :["gov","politics"],
           "minister":["minister","politics"],
           "african":["african"],
          "afghan" :["afghanistan","politics"],
           "europe":["europe"],
           "peace":["peace","politics"],
           "kerry": ["kerry","politics"],
            "syria":["politics"],
              "unsg":['united','nations' ,'secretary' ,'general','politics']
    
          }

In [0]:
def map_words(X_) :
  for i in range(len(X_)) :
    for j in range(len(X_[i])) :

      del_ = []
      add_ = []

      for mapp in dic_map :
        if mapp in X_[i][j] :
          del_ += [X_[i][j]]
          add_ +=  dic_map[mapp]
          break

      for elm in del_ :
        X_[i].remove(elm)

      X_[i]+=add_
    X_[i] = [j for j in X_[i] if len(j)>1]

In [0]:
"""for i in range(len(train_tokens))  :
  l = []
  for w in train_tokens[i]:
    if w not in glove_small :
      l+=[w]
  for elmt in l :
    train_tokens[i].remove(elmt)
      
for i in range(len(test_tokens))  :
  l = []
  for w in test_tokens[i]:
    if w not in glove_small :
      l+=[w]
  for elmt in l :
    test_tokens[i].remove(elmt)"""

In [0]:
GLOVE_PATH = "gdrive/My Drive/glove.twitter.27B.200d.txt"

encoding="utf-8"
    
glove_small = {}

with open(GLOVE_PATH, "rb") as infile:
    for line in infile:
        parts = line.split()
        word = parts[0].decode(encoding)
        nums=np.array(parts[1:], dtype=np.float32)
        glove_small[word] = nums

In [0]:
"""
Now >> Classifiers using vector embeddings. 
We will implement an embedding vectorizer - a counterpart of CountVectorizer and TfidfVectorizer - 
that is given a word -> vector mapping and vectorizes texts by taking the mean of all the vectors corresponding to individual words.
"""

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=200
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

    
# tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        if len(word2vec)>0:
            self.dim=200
        else:
            self.dim=0
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf, 
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])
    
        return self
    
    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [0]:
# Extra Trees classifier is almost universally great, let's stack it with our embeddings


etree_glove_small = Pipeline([("glove vectorizer", MeanEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])


etree_glove_small_tfidf = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])


rfc_glove_small_tfidf = Pipeline([("glove vectorizer", TfidfEmbeddingVectorizer(glove_small)), 
                       ("rfc", RandomForestClassifier(n_estimators=200)) ])


all_models = [
    ("rfc_glove_small_tfidf", rfc_glove_small_tfidf),
    ("glove_small", etree_glove_small),
    ("glove_small_tfidf", etree_glove_small_tfidf),
    
    
]

In [61]:
unsorted_scores = [(name, cross_val_score(model, X, y, cv=5).mean()) for name, model in all_models]
scores = sorted(unsorted_scores, key=lambda x: -x[1])

print (tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

model                    score
---------------------  -------
glove_small             0.9358
glove_small_tfidf       0.9333
rfc_glove_small_tfidf   0.9281
svm_glove_small_tfidf   0.5346


In [0]:
def find_s2(str_,int_) :
  l=[]
  for i in range(len(X_)) :
    for j in X_[i] :
      if str_ in j :
        if int_ == 1 :
          if j == str_ :
             l+=[j]      
        else : 
            l+=[j]
  return l

In [194]:
find_s(X_,"ucl",0)

angelahucles  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
angelahucles  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
angelahucles  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
angelahucles  >>>>>  Politics
angelahucles  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
nuclear  >>>>>  Politics
maximucle  >>>>>  Sports
angelahucles  >>>>>  Politics
nuclear  >>>>>  Politics
angelahucles  >>>>>  Politics
nuclear  >>>>> 
nurbanuclk  >>>>> 
nuclear  >>>>> 
nuclear  >>>>> 
nuclear  >>>>> 


In [0]:
vocab = []
for i in X_ :
  for j in i :
    if j not in vocab :
      vocab += [j]

In [1]:
for i in  vocab :
  if "\'" in i :
    print(i)

NameError: ignored

In [0]:
sorted(vocab)

In [0]:
for i in X_ :
  for j in i :
    if len(j) > 3 :
      l = find_s2(j,0)
      if len(l)> 5 :
        print(l)